In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==2.2
!pip uninstall keras
!pip install keras==2.1.2
!pip uninstall numpy
!pip install numpy==1.19

Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.2+zzzcolab20220719082949.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220719082949
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 516.2 MB 4.7 kB/s 
     |████████████████████████████████| 3.0 MB 58.0 MB/s 
     |████████████████████████████████| 454 kB 73.4 MB/s 
     |████████████████████████████████| 26.1 MB 1.2 MB/s 
     |███████

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
#import packages

import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
import tensorflow as tf
from sklearn.model_selection import train_test_split
import unicodedata
import io
import time



Using TensorFlow backend.


In [ ]:
path_to_file = "/content/drive/MyDrive/Dataset/Train_new.txt"
# Preview the dataset
lines = io.open(path_to_file, encoding='utf-8').read().strip().split('\n')

[[w for w in l.split('\t')[:2]]  for l in lines[:5]]

[['عليك مراجعه جراح عام وعمل صوره رنين مغناطيسي للبطن والتاكد من وضع القنوات الصفراويه احيانا يحصل الالم بعد عمليه المنظار بسبب وجود الهواء في البطن ويزول مع الوقت',
  'اللي بيحس بالم بعد عمليه استئصال مراره سببه ايه بيكون الم في منطقه المراره وفي اسفل الجهه اليمنى من البطن'],
 ['لا علاقه للسكر او الكلسترول بهذه الالام اما باقي فقد تمت الاجابه عنه في نفس المكرر',
  'قريبتي تشتكي من الام اسفل الظهر بقرب عظمه الحوض وتشعر ايضا بنغزات في الثدي مااسباب هذا الالم وهل للسكر والكلسترول سبب في هذا'],
 ['النهايات العصبيه في الاطراف العلويه والسفليه دقيقه وعند البعض تتاثر بالبروده وعند اخرين بالسخونه وخاصه لدى مرضى السكري او من يعانون من اعتلال عصبي محيطي اذا كانت الحاله مزعجه بشكل كبير يمكن عمل تخطيط اعصاب للاطراف العلويه للتاكد من عدم وجود انضغاط عصبي في اليدين',
  'اشعر بتنميل في اطراف الاصابع عندما تلامس المياه وخصوصا المياه الساخنه فما السبب'],
 ['قد يكون ما تعاني منه هو حب الشباب الدهني ولكن لابد من الفحص السرير لتاكيد التشخيص',
  'اريد الاستفسار عن الحبوب على الجسم على الظهر و الصدر و اليد

In [ ]:
word_pairs = [[w for w in l.split('\t')[:2]]  for l in lines]

In [ ]:
texts_1 = [word_pairs[i][0] for i in range(len(word_pairs))] #A
texts_2 = [word_pairs[i][-1] for i in range(len(word_pairs))] #Q

In [ ]:
#PreTreatment

def preTreatement(w):
  
  w = w.lower().strip()

  # unicode to ascii
  w = ''.join(c for c in unicodedata.normalize('NFD', w) if unicodedata.category(c) != 'Mn')
  #delete extra white space at the end & begining
  w = w.strip()

  # add the start and end tokens
  w = '<start> ' + w + ' <end>'
    
  return w

In [ ]:
# making the pre treatement to the sentences, and return the dataset in the format : answers, questions
def extractSentences():

  answer = [preTreatement(w) for w in texts_1]
  question = [preTreatement(w) for w in texts_2]


  return answer, question

In [ ]:
#Tokenization

def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')

  return tensor, lang_tokenizer

In [ ]:
# Load the dataset + Tokenization for the inputs and outputs
#input= Answer
#target= Question
def loadSentences():
    
  # creating cleaned input, output pairs
  inputLang, targetLang = extractSentences()

  inputTensor, inputLang_tokenizer = tokenize(inputLang)
  targetTensor, targetLang_tokenizer = tokenize(targetLang)

  return inputTensor, targetTensor, inputLang_tokenizer, targetLang_tokenizer

In [ ]:
# Load input and output tensors

inputTensor, targetTensor, inputLang, targetLang = loadSentences()

# Calculate max_length of the target tensors
maxLengthTarget, maxLengthInput = targetTensor.shape[1], inputTensor.shape[1]

In [ ]:
# Creating training and validation sets using an 80-20 split (you must also leave another 20% aside for the test ... we have left this 20% in a separate file)

#inputTensor_train, inputTensor_val, targetTensor_train, targetTensor_val = train_test_split(inputTensor, targetTensor, test_size=0.2)
inputTensor_train = inputTensor
targetTensor_train = targetTensor


In [ ]:
#Convert from index to word

def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [ ]:
#show an example
#input= Answer
#target= Question
print ("Input (ANSWER); index to word mapping")
convert(inputLang, inputTensor_train[1])
print ()
print ("Target (QUESTION); index to word mapping")
convert(targetLang, targetTensor_train[1])

Input (ANSWER); index to word mapping
1 ----> <start>
8 ----> لا
86 ----> علاقه
4805 ----> للسكر
7 ----> او
4806 ----> الكلسترول
903 ----> بهذه
477 ----> الالام
64 ----> اما
1976 ----> باقي
193 ----> فقد
1201 ----> تمت
391 ----> الاجابه
584 ----> عنه
4 ----> في
392 ----> نفس
8402 ----> المكرر
2 ----> <end>

Target (QUESTION); index to word mapping
1 ----> <start>
6662 ----> قريبتي
2516 ----> تشتكي
3 ----> من
114 ----> الام
95 ----> اسفل
167 ----> الظهر
8592 ----> بقرب
1161 ----> عظمه
781 ----> الحوض
3129 ----> وتشعر
246 ----> ايضا
3433 ----> بنغزات
4 ----> في
234 ----> الثدي
6663 ----> مااسباب
18 ----> هذا
74 ----> الالم
17 ----> وهل
3130 ----> للسكر
8593 ----> والكلسترول
66 ----> سبب
4 ----> في
18 ----> هذا
2 ----> <end>


In [ ]:
#Definition of hyper Parameters

bufferSize = len(inputTensor_train)
batchSize = 64
stepsPerEpoch = len(inputTensor_train)//batchSize
embeddingDimension = 256
units = 1024
vocabInputSize = len(inputLang.word_index)+1
vocabTargetSize = len(targetLang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((inputTensor_train, targetTensor_train)).shuffle(bufferSize)
dataset = dataset.batch(batchSize, drop_remainder=True)

#example of input and target batches

exampleInputBatch, exampleTargetBatch = next(iter(dataset))
exampleInputBatch.shape, exampleTargetBatch.shape

(TensorShape([64, 52]), TensorShape([64, 52]))

In [ ]:
# Encoder class

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embeddingDimension, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embeddingDimension)
    self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocabInputSize, embeddingDimension, units, batchSize)

sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(exampleInputBatch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 52, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [ ]:
#Attention mechanism class

class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    
    query_with_time_axis = tf.expand_dims(query, 1)
    score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))

    attention_weights = tf.nn.softmax(score, axis=1)

    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batchSize, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batchSize, sequence_length, 1) (64, 52, 1)


In [ ]:
#Decoder class

class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embeddingDimension, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embeddingDimension)
    self.gru = tf.keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)
    

    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    
    context_vector, attention_weights = self.attention(hidden, enc_output)

    x = self.embedding(x)
    
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    output, state = self.gru(x)

    output = tf.reshape(output, (-1, output.shape[2]))
    
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocabTargetSize, embeddingDimension, units, batchSize)

sample_decoder_output, _, _ = decoder(tf.random.uniform((batchSize, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batchSize, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batchSize, vocab size) (64, 27057)


In [ ]:
print(vocabTargetSize)

27057


In [ ]:
print(vocabInputSize)

24976


In [ ]:
#Define the optimizer and the loss function

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
#Checkpoints 

checkpoint_dir = '/content/drive/MyDrive/train_file/28'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=2)

In [ ]:
# Passing to the inference

# restore the latest checkpoint 
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
#Generate Questions

def evaluate(sentence):

  sentence = preTreatement(sentence)

  inputs = [inputLang.word_index[i] for i in sentence.split(' ') if i in inputLang.word_index.keys()]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=maxLengthInput, padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targetLang.word_index['<start>']], 0)

  for t in range(maxLengthTarget):
        
    predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targetLang.index_word[predicted_id] + ' '

    if targetLang.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)
    
  return result, sentence

In [ ]:
def generate_question(sentence):
    
  result, sentence = evaluate(sentence)
  f_res = result[:-6].rstrip()
  return f_res

In [ ]:
# path to the dataset
df1= pd.read_pickle("/content/drive/MyDrive/Dataset/Test_new.pkl")
df1.set_axis(["Answer", "Question"],axis=1,inplace=True)



In [ ]:
#Execute only once to create separate refrences and candidatess files

for i in range(len(df1)):
    
    reference_file=open("/content/drive/MyDrive/Refrence_28.txt", "a+")
    refrence=df1['Question'][i]
    reference_file.write(str(refrence)+'\n')
    reference_file.close()
    
    candidate_file=open("/content/drive/MyDrive/Candidate_28.txt", "a+")
    candidate_file.write(generate_question(df1['Answer'][i])+'\n')
    candidate_file.close()
   
  



In [ ]:
candidte_file= open("/content/drive/MyDrive/Candidate_28.txt", "r")

for line in candidte_file:
   
    all_candidates = [line.rstrip('\n') for line in candidte_file]
   

In [ ]:
reference_file= open("/content/drive/MyDrive/Refrence_28.txt", "r")
for line in reference_file:
    all_refrences = [line.rstrip('\n') for line in reference_file]
    

In [ ]:
import nltk
nltk.download("punkt")
liste_candidates=[]
for words in all_candidates:
    liste_candidates.append(nltk.word_tokenize(words))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import nltk
nltk.download("punkt")
liste_ref=[]
for words in all_refrences:
    liste_ref.append(nltk.word_tokenize(words))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import nltk
nltk.download("punkt")
liste_refrences=[]
liste_of_liste_refrences=[]
for words in all_refrences:
    liste_refrences.append(nltk.word_tokenize(words))
    liste_of_liste_refrences.append(liste_refrences)
    liste_refrences=[]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Sacre bleu**

In [ ]:
pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 116 kB 33.0 MB/s 


In [ ]:
import sacrebleu

#bleu4
sacrebleu.corpus_bleu(all_candidates, [all_refrences])

BLEU = 10.03 16.3/11.2/10.3/9.9 (BP = 0.858 ratio = 0.867 hyp_len = 38826 ref_len = 44758)

**Torch Metrics**

In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 34.4 MB/s 


In [ ]:

liste_refrences2=[]
liste_of_liste_refrences2=[]
for words in all_refrences:
    liste_refrences2.append(words)
    liste_of_liste_refrences2.append(liste_refrences2)
    liste_refrences2=[]

In [ ]:
from torchmetrics import BLEUScore
#bleu4
metric = BLEUScore()
metric(all_candidates,liste_of_liste_refrences2)

tensor(0.1002)

**BLEU**

In [ ]:
#Bleu-1
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
chencherry = SmoothingFunction()
Bleu_score1= corpus_bleu(liste_of_liste_refrences,liste_candidates, weights=(1,0,0,0), smoothing_function = chencherry.method5)
print(Bleu_score1)

0.41145044297262906


In [ ]:
#Bleu-2
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
chencherry = SmoothingFunction()
Bleu_score2= corpus_bleu(liste_of_liste_refrences,liste_candidates, weights=(0.5, 0.5,))
print(Bleu_score2)

0.11603093294865212


In [ ]:
#Bleu-3
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
chencherry = SmoothingFunction()
Bleu_score3= corpus_bleu(liste_of_liste_refrences,liste_candidates, weights=(0.333, 0.333, 0.334,))
print(Bleu_score3)

0.10598502504894575


In [ ]:
#Bleu-4
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
chencherry = SmoothingFunction()
Bleu_score = corpus_bleu(liste_of_liste_refrences,liste_candidates)
print(Bleu_score)

0.10022999924535102


**Meteor**

**Méthode 1**

In [ ]:
!pip install git+https://github.com/huggingface/evaluate@a45df1eb9996eec64ec3282ebe554061cb366388



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/evaluate (to revision a45df1eb9996eec64ec3282ebe554061cb366388) to /tmp/pip-req-build-jruccqx0
  Running command git clone -q https://github.com/huggingface/evaluate /tmp/pip-req-build-jruccqx0
  Running command git rev-parse -q --verify 'sha^a45df1eb9996eec64ec3282ebe554061cb366388'
  Running command git fetch -q https://github.com/huggingface/evaluate a45df1eb9996eec64ec3282ebe554061cb366388
  Running command git checkout -q a45df1eb9996eec64ec3282ebe554061cb366388
     |████████████████████████████████| 365 kB 22.5 MB/s 
     |████████████████████████████████| 212 kB 71.4 MB/s 
     |████████████████████████████████| 115 kB 70.7 MB/s 
     |████████████████████████████████| 120 kB 69.7 MB/s 
     |████████████████████████████████| 127 kB 72.0 MB/s 
  Created wheel for evaluate: filename=evaluate-0.1.3.dev0-py3-none-any.whl size=55236 sha256=0dce

In [ ]:
#Méthode 1
#FROM :https://huggingface.co/spaces/evaluate-metric/meteor
import evaluate
meteor = evaluate.load('meteor')
predictions = all_candidates
references = all_refrences
results = meteor.compute(predictions=predictions, references=references)
print(results)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


{'meteor': 0.11616491513850566}


**Méthode 2**

In [ ]:
#Méthode 2 (Ma fonction)
from nltk.translate.meteor_score import meteor_score


import numpy as np

def corpus_meteor(expected, predicted):
    meteor_score_sentences_list = list()
    #Pour calculer le score entre chaque paire
    [meteor_score_sentences_list.append(meteor_score(expect, predict)) for expect, predict in zip(expected, predicted)]
    #la moyenne de la somme des scores de chaque paire
    meteor_score_res = np.mean(meteor_score_sentences_list)
    return meteor_score_res

Score_meteor= corpus_meteor(liste_of_liste_refrences,liste_candidates)
print(Score_meteor)

0.11616491513850566


**Rouge**

**Méthode 1**


In [ ]:
#https://github.com/pltrdy/rouge
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import FilesRouge

files_rouge = FilesRouge()
hyp_path='/content/drive/MyDrive/Candidate_28.txt'
ref_path='/content/drive/MyDrive/Refrence_28.txt'
#score total: la moyenne des scores entre chaque paire
scores = files_rouge.get_scores(hyp_path, ref_path, avg=True)
print(scores)

{'rouge-1': {'r': 0.1416801773257511, 'p': 0.2026403721322335, 'f': 0.1540969797109862}, 'rouge-2': {'r': 0.09051456062836127, 'p': 0.10899856895742763, 'f': 0.094455699122403}, 'rouge-l': {'r': 0.13780918910754802, 'p': 0.19454135930585909, 'f': 0.14928038701900248}}


**Méthode 2**

In [ ]:
#https://gist.github.com/jamescalam/eca142d1dd7eda4d1dea5182541077cc#file-rouge_avg-ipynb
from rouge import Rouge
model_out = all_candidates
reference = all_refrences
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

{'rouge-1': {'r': 0.14175867049878477,
  'p': 0.20275263826637877,
  'f': 0.15418235199891472},
 'rouge-2': {'r': 0.09056470719934652,
  'p': 0.1090589559762406,
  'f': 0.09450802914961763},
 'rouge-l': {'r': 0.13788553768877104,
  'p': 0.19464913845228893,
  'f': 0.14936309083452548}}

**Méthode 3**

In [ ]:
#https://github.com/li-plus/rouge-metric
!pip install rouge-metric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 151 kB 30.6 MB/s 


In [ ]:
#creation de liste de lite non tokenisé pour la ref
liste_ref=[]
liste_of_liste_ref=[]
for i in all_refrences:
    liste_ref.append(i)
    liste_of_liste_ref.append(liste_ref)
    liste_ref=[]

In [ ]:
#https://github.com/li-plus/rouge-metric
from rouge_metric import PyRouge


hypotheses = all_candidates #[1,2]
references = liste_of_liste_ref#[[1],[2]]

# Evaluate document-wise ROUGE scores
'''rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=False,
                rouge_w_weight=1.2, rouge_s=False, rouge_su=False, skip_gap=4)'''

rouge = PyRouge(rouge_l=True)
scores = rouge.evaluate(hypotheses, references)
print(scores)

{'rouge-1': {'r': 0.14099389208686308, 'p': 0.17469142358997064, 'f': 0.15604415221745008}, 'rouge-2': {'r': 0.09052464179553243, 'p': 0.09704139263780405, 'f': 0.09366980897598504}, 'rouge-l': {'r': 0.13704700237503206, 'p': 0.16767071018545118, 'f': 0.15082003618314638}}
